In [1]:
import numpy as np
import glob
import os

import endocytosis.contrib.gohlke.tifffile as tifffile
import endocytosis.io.path as path
import endocytosis.io.image.roi.file_reading as file_reading

In [2]:
folder = r"D:\endocytosis_data_subset\2016-12-14 iso\2016-12-14 Sp57 proto 003"
filepaths = glob.glob(folder + os.path.sep + '*.tif')

In [3]:
filepaths

['D:\\endocytosis_data_subset\\2016-12-14 iso\\2016-12-14 Sp57 proto 003\\2016-12-14 Sp57 proto 003.tif']

In [4]:
im = tifffile.TiffFile(filepaths[0])

In [5]:
pathfinder = path.PathFinder(regexp='RoiSetAuto.*', extension='zip')

In [6]:
zip_filenames = [p[0] for p in pathfinder.load(folder)]

In [7]:
folder + os.path.sep + zip_filenames[0]

'D:\\endocytosis_data_subset\\2016-12-14 iso\\2016-12-14 Sp57 proto 003\\RoiSetAuto-100.zip'

In [8]:
roi_reader = file_reading.IJZipReader(regexp=r'.*[0-9](#x)?\.roi$', sep=None)

In [9]:
roi_reader.read(folder + os.path.sep + zip_filenames[0])

In [10]:
for k, v in roi_reader.items():
    print(k, v.centroid)

0030-0030-0964#x {'px': array([729.5, 964.5])}
0031-0031-0965 {'px': array([728.5, 965.5])}
0032-0032-0965 {'px': array([730.5, 965.5])}
0033-0033-0967 {'px': array([729.5, 967.5])}
0034-0034-0967 {'px': array([728.5, 967.5])}
0035-0035-0968 {'px': array([726.5, 968.5])}
0036-0036-0970#x {'px': array([726.5, 970.5])}
0037-0037-0970 {'px': array([726.5, 970.5])}
0038-0038-0969 {'px': array([727.5, 969.5])}
